In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import ArrayType, FloatType, IntegerType, StringType, StructField, StructType
from pyspark.sql.functions import col, size, udf

conf = SparkConf() \
    .set("fs.s3a.awsAccessKeyId", access) \
    .set("fs.s3a.awsSecretAccessKey", secret) \
    .set("fs.s3a.endpoint", "s3.us-east-1.amazonaws.com") \
    .set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("fs.s3a.impl","org.apache.hadoop.fs.s3native.NativeS3FileSystem") \
    .set("com.amazonaws.services.s3.enableV4", "true") \
    .set("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .set("spark.hadoop.fs.s3a.fast.upload","true") \
    .set("spark.sql.parquet.filterPushdown", "true") \
    .set("spark.sql.parquet.compression.codec", "snappy")

spark = SparkSession.builder.appName('data-cleaning').config(conf=conf).getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1619713790525_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import sparknlp
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,
                                LemmatizerModel, StopWordsCleaner,
                               SentenceDetector)
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
filename = ''
df = spark.read.parquet(filename).drop('geo', 'coordinates', 'place', 'Unnamed: 0', 'Unnamed: 0.1', '__index_level_0__', 'retweet_count', 'favorite_count')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
import preprocessor as p

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [157]:
def tweetPreprocessor(text):
    p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.NUMBER, p.OPT.MENTION)
    text = p.clean(text)
    return text
tweetPreprocessor_udf = udf(tweetPreprocessor, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
def extractHashtag(text):
    result = []
    parse = p.parse(text)

    if parse.hashtags == None:
        return result
    
    for i in range(len(parse.hashtags)):
        result.append(parse.hashtags[i].match[1:])
    return result

extractHashtag_udf = udf(extractHashtag, ArrayType(StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
df = df.withColumn('clean', tweetPreprocessor_udf('full_text'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
# The nltk stopword won't download onto emr without some setup so it's just easier to copy and paste
stopwords_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'youre', 'youve', 'youll', 'youd', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'shes', 'her', 'hers', 'herself', 'it', 'its', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'thatll', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'dont', 'should', 'shouldve', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'arent', 'couldn', 'couldnt', 'didn', 'didnt', 'doesn', 'doesnt', 'hadn', 'hadnt', 'hasn', 'hasnt', 'haven', 'havent', 'isn', 'isnt', 'ma', 'mightn', 'mightnt', 'mustn', 'mustnt', 'needn', 'neednt', 'shan', 'shant', 'shouldn', 'shouldnt', 'wasn', 'wasnt', 'weren', 'werent', 'won', 'wont', 'wouldn', 'wouldnt', 'like', 'hes', 'let', 'lot', 'ok', 'yes', 'u', 'im', 'y', 'am', 'ive', 'am', 'k']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
documentAssembler = DocumentAssembler() \
    .setInputCol('clean') \
    .setOutputCol('document') \
    .setCleanupMode('shrink_full')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['normalized']) \
    .setOutputCol('normalized_no_stop') \
    .setCaseSensitive(False) \
    .setStopWords(stopwords_list)

lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized_no_stop']) \
    .setOutputCol('lemma')

finisher = Finisher() \
     .setInputCols(['lemma']) \
     .setCleanAnnotations(True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]

In [163]:
pipeline = Pipeline() \
     .setStages([
           documentAssembler,
           tokenizer,
           normalizer,
           stopwords_cleaner,
           lemmatizer,
           finisher
     ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [164]:
df = pipeline.fit(df).transform(df)
df = df.drop('clean').withColumnRenamed('finished_lemma', 'clean_text')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------------------+--------------------+
|         created_at|                 id|           full_text|          clean_text|
+-------------------+-------------------+--------------------+--------------------+
|2020-10-21 05:50:49|1318791833992929282|@SomeWelder @Vick...|[hasty, cowboy, f...|
|2020-10-26 11:05:00|1320682836987416576|Biden was pretty ...|[biden, pretty, m...|
|2020-10-06 05:42:24|1313353897508048903|Biden says people...|[biden, say, peop...|
|2020-10-22 23:21:17|1319418580476317698|@peterdaou And Bi...|[biden, want, abo...|
|2020-10-14 12:26:22|1316354661662810112|@KagcaravanLisa H...|[always, say, itb...|
+-------------------+-------------------+--------------------+--------------------+
only showing top 5 rows

In [166]:
df = df.where(size(col('clean_text')) > 5)
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------------------+--------------------+
|         created_at|                 id|           full_text|          clean_text|
+-------------------+-------------------+--------------------+--------------------+
|2020-10-21 05:50:49|1318791833992929282|@SomeWelder @Vick...|[hasty, cowboy, f...|
|2020-10-26 11:05:00|1320682836987416576|Biden was pretty ...|[biden, pretty, m...|
|2020-10-06 05:42:24|1313353897508048903|Biden says people...|[biden, say, peop...|
|2020-10-14 12:26:22|1316354661662810112|@KagcaravanLisa H...|[always, say, itb...|
|2020-10-02 07:41:04|1311934210027016192|@PauleyP @DebraMe...|[we, forget, dont...|
|2020-10-18 08:51:15|1317750078442975237|This. Is great ne...|[great, newsi, en...|
|2020-10-07 00:52:37|1313643358930296832|@realDonaldTrump ...|[hell, go, fbi, c...|
|2020-10-30 06:38:13|1322065253245480961|@FluffyTrader The...|[election, uncert...|
|2020-10-19 13:28:10|1318182154350825478|@politvidchannel ...|[that, bad, th

In [167]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [168]:
sentiment_schema = StructType([
    StructField('pos', FloatType(), False),
    StructField('neg', FloatType(), False),
    StructField('neu', FloatType(), False)
])

def sentimentAnalysis(bow_text):
    text = ' '.join(bow_text)
    scores = SentimentIntensityAnalyzer().polarity_scores(text)
    return (scores['pos'], scores['neg'], scores['neu'])

sentimentAnalysis_udf = udf(sentimentAnalysis, sentiment_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [169]:
df = df.withColumn('sentiment', sentimentAnalysis_udf('clean_text'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
def classifySentiment(scores):
    pos = scores[0]
    neg = scores[1]
    if pos > neg:
        return 1
    elif pos < neg:
        return -1
    else:
        return 0

classifySentiment_udf = udf(classifySentiment, IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [171]:
df = df.withColumn('sentiment_class', classifySentiment_udf('sentiment'))
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4625

In [172]:
positive = df.where(col('sentiment_class') == 1)
positive.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1962

In [173]:
negative = df.where(col('sentiment_class') == -1)
negative.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1930

In [ ]:
hdfs_write = ''
positive.write.mode('overwrite').parquet(hdfs_write)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…